This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [21]:
#! pip install python-dotenv --user
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

#print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [4]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [5]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://data.nasdaq.com/api/v3/datasets/WIKI/FB/data.json?api_key=3zfYxfALV2aMmeyVKcf1"
r = requests.get(url)
json_data = r.json()

In [6]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


# This commented out part will print everything
#for k in json_data["dataset_data"].keys():
#    print(k + ': ', json_data["dataset_data"][k])

print(json_data["dataset_data"].keys())
print(json_data["dataset_data"]["data"][0:2])

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])
[['2018-03-27', 156.31, 162.85, 150.75, 152.19, 76787884.0, 0.0, 1.0, 156.31, 162.85, 150.75, 152.19, 76787884.0], ['2018-03-26', 160.82, 161.1, 149.02, 160.06, 125438294.0, 0.0, 1.0, 160.82, 161.1, 149.02, 160.06, 125438294.0]]


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

###### 1.  Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

*Hint: We are looking for the AFX_X data on the datasets/FSE/ dataset.*

In [7]:
#https://data.nasdaq.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}?&{start_date = <YYYY-MM-DD>}&{end_date = <YYYY-MM-DD>}&{api_key=<API_KEY>}
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?&start_date=2017-01-01&end_date=2017-12-31&api_key=3zfYxfALV2aMmeyVKcf1"
r = requests.get(url)
r

<Response [200]>

###### 2. Convert the returned JSON object into a Python dictionary.

In [8]:
json_data = r.json()

In [9]:
print(json_data["dataset_data"].keys())
print(json_data["dataset_data"]["data"][0:2])

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])
[['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None], ['2017-12-28', 51.65, 51.82, 51.43, 51.6, None, 40660.0, 2099024.0, None, None, None]]


In [10]:
json_data["dataset_data"]["column_names"] # Looking for the index of each value in the lists

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

###### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [11]:
open_index = json_data["dataset_data"]["column_names"].index("Open")
opens = []

for i in json_data['dataset_data']["data"]:
    opens.append(i[open_index])

opens = [num for num in opens if isinstance(num, (int, float))] # Some values are "None"

print("Max Opens:", max(opens))
print("Min Opens:", min(opens))

Max Opens: 53.11
Min Opens: 34.0


###### 4. What was the largest change in any one day (based on High and Low price)?

In [12]:
high_index = json_data["dataset_data"]["column_names"].index("High")
low_index = json_data["dataset_data"]["column_names"].index("Low")

changes = []


for i in json_data['dataset_data']["data"]:
    changes.append(i[high_index] - i[low_index])
    
max(changes)

2.8100000000000023

###### 5. What was the largest change between any two days (based on Closing Price)?

In [15]:
# Get create a list of close values like the previous steps
close_index = json_data["dataset_data"]["column_names"].index("Close")

closes = []

for i in json_data['dataset_data']["data"]:
    closes.append(i[close_index])

closes[0:5]

[51.76, 51.6, 51.82, 51.32, 51.4]

In [16]:
close_changes = []

for i in range(1, len(closes)):
    
    if i == 0:
        pass
    else:
        close_changes.append(abs(closes[i] - closes[i - 1])) #append the absolute value of each close minus the previous close
    
max(close_changes)

2.559999999999995

###### 6. What was the average daily trading volume during this year?

In [17]:
trade_volume_index = json_data["dataset_data"]["column_names"].index("Traded Volume")

trade_volumes = []

for i in json_data['dataset_data']["data"]:
    trade_volumes.append(i[trade_volume_index])
    
sum(trade_volumes) / len(trade_volumes)

89124.33725490196

###### (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [18]:
def median(list):
    list_nums = [n for n in list if isinstance(n, (int, float))] # filter to only integers and floats
    
    list_nums.sort() # sort in ascending order
    
    if len(list)%2 == 1: # check if list lenth is odd
        median = list[len(list)//2] # take the middle value
    elif len(list)%2 == 0: #check if list length is even
        median = (list[(len(list)//2) - 1] + list[(len(list)//2)])/2 #take the midpoint between the two middle values
        
    return median

In [19]:
# Testing median function
print("Even length list test:", median([21, 22, 23, 24, 25, 26]))
print("Odd length list test:", median([21, 22, 23, 24, 25, 26, 27]))

Even length list test: 23.5
Odd length list test: 24


In [20]:
# Actual output
print("Trade Volume Median:", median(trade_volumes))

Trade Volume Median: 134965.0
